In [84]:
from pyspark.sql.functions import col,udf,collect_list,array_contains

from pyspark.ml.feature import IDF, CountVectorizer,StringIndexer
from pyspark.ml.clustering import LDA
from pyspark.ml.linalg import SparseVector, VectorUDT,DenseVector

import numpy as np
import pandas as pd
import random
from metric_learn import SDML

from sklearn import preprocessing

## 1.Load data to DataFrame

In [2]:
dias_df = spark.read.csv("../../mimic3/data/DIAGNOSES_ICD.csv", header=True, mode="DROPMALFORMED")
dic_ICD_df = spark.read.csv("../../mimic3/data/D_ICD_DIAGNOSES.csv", header=True, mode="DROPMALFORMED")

## 2.Data Pre-processing 

In [3]:
dias_df.createOrReplaceTempView("diagnosis")

#Fitering the unspecified disgnosis codes
filteredDiags = spark.sql("SELECT SUBJECT_ID,ICD9_CODE FROM diagnosis WHERE ICD9_CODE not in ('', '4019','7793','2724','2449')")

### 2.1 Extract frequent diagnosis codes

In [4]:
#Choosing top 500 frequent diagnosis codes.
topDiags = filteredDiags.groupBy("ICD9_CODE").count().sort(col("count").desc()).select("ICD9_CODE").limit(500)

### 2.2 Extract patients who had the requent diagnosis codes

In [5]:
#Inner join to get patients who had the top 500 diagnosis codes.
top_freq_pats = filteredDiags.join(topDiags, filteredDiags.ICD9_CODE == topDiags.ICD9_CODE, "inner").\
                drop(topDiags.ICD9_CODE)

### 2.3 Aggregate diagnosis codes list through grouping by "SUBJECT_ID"

In [7]:
pats_dias = top_freq_pats.groupBy("SUBJECT_ID").agg(collect_list("ICD9_CODE"))
pats_dias = pats_dias.select(col("SUBJECT_ID"),col("collect_list(ICD9_CODE)").alias("codes"))

### 2.4 Encode patients and diagnosis codes

In [8]:
#index subject_id to label
indexer = StringIndexer(inputCol="SUBJECT_ID", outputCol="label")
indexed = indexer.fit(pats_dias).transform(pats_dias)

#terms' count vector
vector = CountVectorizer(inputCol="codes", outputCol="tf_features")
countVect = vector.fit(indexed)  

#get vocaulary
vocabs = countVect.vocabulary

### 2.5 Frequency of diagnosis codes

In [9]:
freqVect = countVect.transform(indexed)

In [10]:
freqVect.printSchema()

root
 |-- SUBJECT_ID: string (nullable = true)
 |-- codes: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- label: double (nullable = true)
 |-- tf_features: vector (nullable = true)



### 2.6 TF-IDF of diagnosis codes

In [11]:
# dataset = freqVect.select(col("label"),col("features").alias("rawFeatures"))
idf = IDF(inputCol="tf_features", outputCol="tfidf_features")
idfModel = idf.fit(freqVect)
tf_idfVect = idfModel.transform(freqVect)

In [12]:
# tf_idfVect = tf_idfVect.selectExpr("features as TFIDF_features")
tf_idfVect.printSchema()

root
 |-- SUBJECT_ID: string (nullable = true)
 |-- codes: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- label: double (nullable = true)
 |-- tf_features: vector (nullable = true)
 |-- tfidf_features: vector (nullable = true)



### 2.7 OneHot of diagnosis codes

In [13]:
def to_sparse(c):
    def to_sparse_(v):
#         if isinstance(v, SparseVector):
#             return v
        vs = (v.toArray()>0)*1
        nonzero = np.nonzero(vs)[0]
        return SparseVector(v.size, nonzero, vs[nonzero])
    return udf(to_sparse_, VectorUDT())(c)

oneHot_freqVect = tf_idfVect.withColumn("oneHot_features",to_sparse(tf_idfVect.tf_features))

### 2.8 AutoEncoders of diagnosis codes

In [16]:
from keras.layers import Input, Dense
from keras.models import Model

# this is the size of our encoded representations
encoding_dim = 32 

# this is our input placeholder
input_patient = Input(shape=(500,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_patient)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(500, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_patient, decoded)

Using TensorFlow backend.


In [17]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

#### Converting Spark ML Vector to Numpy Array

In [18]:
#Converting to Panda’s dataframe
array_features = oneHot_freqVect.select('oneHot_features').toPandas()

In [19]:
#Convert Sparse Vector to Matrix
dataset = array_features['oneHot_features'].apply(lambda x : x.toArray()).as_matrix().reshape(-1,1)

#Flatten using apply_along_axis
features = np.apply_along_axis(lambda x : x[0], 1, dataset)

In [20]:
features

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(features, test_size=0.20, random_state=42)

In [22]:
autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))

Train on 36904 samples, validate on 9227 samples
Epoch 1/50
36904/36904 [==============================] - 1s 30us/step - loss: 0.6220 - val_loss: 0.4284
Epoch 2/50
36904/36904 [==============================] - 1s 27us/step - loss: 0.2050 - val_loss: 0.1150
Epoch 3/50
36904/36904 [==============================] - 1s 27us/step - loss: 0.1042 - val_loss: 0.0969
Epoch 4/50
36904/36904 [==============================] - 1s 27us/step - loss: 0.0950 - val_loss: 0.0921
Epoch 5/50
36904/36904 [==============================] - 1s 28us/step - loss: 0.0914 - val_loss: 0.0893
Epoch 6/50
36904/36904 [==============================] - 1s 28us/step - loss: 0.0889 - val_loss: 0.0872
Epoch 7/50
36904/36904 [==============================] - 1s 29us/step - loss: 0.0868 - val_loss: 0.0854
Epoch 8/50
36904/36904 [==============================] - 1s 28us/step - loss: 0.0851 - val_loss: 0.0837
Epoch 9/50
36904/36904 [==============================] - 1s 27us/step - loss: 0.0835 - val_loss: 0.0823
Epoch 

In [23]:
# this model maps an input to its encoded representation
encoder = Model(input_patient, encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [24]:
# encode and decode some digits
# note that we take them from the *test* set
encoded_patients = encoder.predict(X_test)
# decoded_patients = decoder.predict(encoded_patients)

In [25]:
oneHot_freqVect.printSchema()

root
 |-- SUBJECT_ID: string (nullable = true)
 |-- codes: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- label: double (nullable = true)
 |-- tf_features: vector (nullable = true)
 |-- tfidf_features: vector (nullable = true)
 |-- oneHot_features: vector (nullable = true)



In [26]:
oneHot_freqVect.show()

+----------+--------------------+-------+--------------------+--------------------+--------------------+
|SUBJECT_ID|               codes|  label|         tf_features|      tfidf_features|     oneHot_features|
+----------+--------------------+-------+--------------------+--------------------+--------------------+
|     10096|[4260, 41401, 530...| 9643.0|(500,[2,7,39,51,2...|(500,[2,7,39,51,2...|(500,[2,7,39,51,2...|
|     10351|[7661, V290, V293...|19592.0|(500,[9,10,32,267...|(500,[9,10,32,267...|(500,[9,10,32,267...|
|     10436|         [431, 3314]|28247.0|(500,[78,212],[1....|(500,[78,212],[3....|(500,[78,212],[1....|
|      1090|[V3001, V502, V05...|43282.0|(500,[9,10,32,52]...|(500,[9,10,32,52]...|(500,[9,10,32,52]...|
|     11078|[41071, 41401, 42...|44246.0|(500,[2,4,28,30,1...|(500,[2,4,28,30,1...|(500,[2,4,28,30,1...|
|     11332|[99811, 48283, 42...| 7917.0|(500,[0,1,63,287,...|(500,[0,1,63,287,...|(500,[0,1,63,287,...|
|     11563|[51881, 03849, 57...|  930.0|(500,[4,5,15,1

### 2.9 Topic Model of diagnosis codes

In [27]:
dataset = oneHot_freqVect.select(col("label"),col("tf_features").alias("features"))

#train LDA model
lda_mimic = LDA(k=10, maxIter=10)
model_mimic = lda_mimic.fit(dataset)

In [28]:
# Describe topics.
tf_topics = model_mimic.describeTopics(10)

In [29]:
#get distribution matrix of documents to topics
docTopics = model_mimic.transform(dataset)

## 3. Clustering

### 3.1 KMeans

1. Features from AutoEncoder

In [30]:
from sklearn.cluster import SpectralClustering,KMeans,MiniBatchKMeans,AgglomerativeClustering,Birch

In [45]:
#Get all patients' autoencoders
encoded_patients = encoder.predict(features)
ae_patients = pd.DataFrame(encoded_patients)

In [46]:
#add a new column as subject_id
ae_patients['SUBJECT_ID'] = oneHot_freqVect.select('SUBJECT_ID').toPandas()

In [41]:
#Check if there are some patients have the two diagnosis codes.
patient2  = oneHot_freqVect.where(array_contains(oneHot_freqVect.codes,"725"))
patient1  = oneHot_freqVect.where(array_contains(oneHot_freqVect.codes,"77181"))
patients = patient1.join(patient2, patient1.SUBJECT_ID == patient2.SUBJECT_ID, 'inner')
patients.count(),patient1.count(),patient2.count()

(0, 222, 225)

In [42]:
#convert spark dataframe to pandas dataframe
patient1_pd  = patient1.select('SUBJECT_ID').toPandas()
patient2_pd  = patient2.select('SUBJECT_ID').toPandas()

In [47]:
ae_patients.shape

(46131, 33)

In [49]:
#get patients' autoencoder codes
profs1 = pd.merge(ae_patients,patient1_pd,how='inner',on='SUBJECT_ID').drop(['SUBJECT_ID'], axis=1)
profs2 = pd.merge(ae_patients,patient2_pd,how='inner',on='SUBJECT_ID').drop(['SUBJECT_ID'], axis=1)
profs = pd.concat([profs1,profs2],axis=0)

#Assign label to each patient
pat1 = pd.DataFrame({'label':np.ones(patient1_pd.shape[0])})
pat2 = pd.DataFrame({'label':np.zeros(patient2_pd.shape[0])})
labels = pd.concat([pat1,pat2],axis=0)
labels.label = labels.label.astype(int)

In [50]:
clusters = KMeans(n_clusters=2).fit_predict(profs)

In [52]:
labels['pred'] = clusters

In [53]:
from sklearn import metrics

In [54]:
metrics.adjusted_mutual_info_score(labels.pred, labels.label),\
metrics.adjusted_rand_score(labels.pred, labels.label),\
metrics.homogeneity_completeness_v_measure(labels.pred, labels.label),\
metrics.fowlkes_mallows_score(labels.pred, labels.label)

(0.0035309639150432623,
 0.003047740592616972,
 (0.0084510646969259, 0.005151601020241131, 0.006401173772852876),
 0.610738698304978)

In [55]:
samples = labels[labels['label']==1]

In [93]:
def getSample(labels):
    sim_samples_list = []
    uniq_lables = labels.label.unique()
    for label in range(uniq_lables.size):
        samples = labels[labels['label']==uniq_lables[label]]
        uniq_cluster = samples.pred.unique()
        if uniq_cluster.size == 1: continue
        for first_cluster_index in range(uniq_cluster.size-1):
            first_cluster_samples = samples[samples['pred']==uniq_cluster[first_cluster_index]]
            first_cluster_samples.is_copy = False
            first_cluster_samples['id'] = first_cluster_samples.index.tolist()
            for second_cluster_index in range(first_cluster_index+1,uniq_cluster.size):
                second_cluster_samples = samples[samples['pred']==uniq_cluster[second_cluster_index]]
                second_cluster_samples.is_copy = False
                second_cluster_samples['id'] = second_cluster_samples.index.tolist()
                first_cluster_samples['key'] = 1
                second_cluster_samples['key'] = 1
                df = pd.merge(first_cluster_samples, second_cluster_samples, on='key')
                del df['key']
                sim_samples_list.append(df)
    if(len(sim_samples_list)> 0):
        sim_samples =  pd.concat(sim_samples_list, axis=0)    
        sim_samples = sim_samples.reset_index()
        index = random.randint(0,sim_samples.shape[0])
        return sim_samples.iloc[[index]]
    else:
        return 'null';

In [106]:
sim_sample = getSample(labels)

In [107]:
sim_sample.describe()

,index,label_x,pred_x,id_x,label_y,pred_y,id_y
count,1.0,1.0,1.0,1.0,1.0,1.0,1.0
mean,5849.0,1.0,0.0,171.0,1.0,1.0,31.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,5849.0,1.0,0.0,171.0,1.0,1.0,31.0
25%,5849.0,1.0,0.0,171.0,1.0,1.0,31.0
50%,5849.0,1.0,0.0,171.0,1.0,1.0,31.0
75%,5849.0,1.0,0.0,171.0,1.0,1.0,31.0
max,5849.0,1.0,0.0,171.0,1.0,1.0,31.0


In [108]:
def metricLearning(samples, df_data):
    
     #get unique row ids
    rowIDLIst = pd.concat([samples.id_x,samples.id_y],axis = 0).unique().tolist()

    #connectivity graph
    cmatrix = np.zeros([len(rowIDLIst),len(rowIDLIst)])
    
    for index,row in samples.iterrows():
        cmatrix[rowIDLIst.index(row.id_x)][rowIDLIst.index(row.id_y)] = 1
        cmatrix[rowIDLIst.index(row.id_y)][rowIDLIst.index(row.id_x)] = 1
        
#     print(cmatrix)
    trainedData = []
    for rid in rowIDLIst:
        row = df_data.iloc[[rid]]
        trainedData.append(row) 
    trainedData = pd.concat(trainedData,axis = 0).as_matrix()   
    
#     print(trainedData)
    metric = SDML(use_cov=False).fit(trainedData, cmatrix)  
    newData = metric.transform(df_data) 
    return newData

In [109]:
df_data = profs
for i in range(10):
    clusters = KMeans(n_clusters=2).fit_predict(df_data)
    print(np.count_nonzero(clusters))
    labels['pred'] = clusters
    print(metrics.adjusted_mutual_info_score(labels.pred, labels.label))
    sim_sample = getSample(labels)
    if(isinstance(sim_sample, pd.DataFrame)):
        df_data = metricLearning(sim_sample,df_data)
        newData = preprocessing.StandardScaler().fit_transform(df_data)
        df_data = pd.DataFrame(newData)

67
0.0035309639150432623
66
0.0028395333133259437


/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function pinvh is deprecated; sklearn.utils.extmath.pinvh was deprecated in version 0.19 and will be removed in 0.21. Use scipy.linalg.pinvh instead.
  warnings.warn(msg, category=DeprecationWarning)
/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function pinvh is deprecated; sklearn.utils.extmath.pinvh was deprecated in version 0.19 and will be removed in 0.21. Use scipy.linalg.pinvh instead.
  warnings.warn(msg, category=DeprecationWarning)
/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function pinvh is deprecated; sklearn.utils.extmath.pinvh was deprecated in version 0.19 and will be removed in 0.21. Use scipy.linalg.pinvh instead.
  warnings.warn(msg, category=DeprecationWarning)
/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: Deprecat

66
0.0028395333133259437
67
0.0035309639150432623


/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function pinvh is deprecated; sklearn.utils.extmath.pinvh was deprecated in version 0.19 and will be removed in 0.21. Use scipy.linalg.pinvh instead.
  warnings.warn(msg, category=DeprecationWarning)
/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function pinvh is deprecated; sklearn.utils.extmath.pinvh was deprecated in version 0.19 and will be removed in 0.21. Use scipy.linalg.pinvh instead.
  warnings.warn(msg, category=DeprecationWarning)
/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function pinvh is deprecated; sklearn.utils.extmath.pinvh was deprecated in version 0.19 and will be removed in 0.21. Use scipy.linalg.pinvh instead.
  warnings.warn(msg, category=DeprecationWarning)
/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: Deprecat

67
0.0035309639150432623
381
0.002839533313325943


/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function pinvh is deprecated; sklearn.utils.extmath.pinvh was deprecated in version 0.19 and will be removed in 0.21. Use scipy.linalg.pinvh instead.
  warnings.warn(msg, category=DeprecationWarning)
/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function pinvh is deprecated; sklearn.utils.extmath.pinvh was deprecated in version 0.19 and will be removed in 0.21. Use scipy.linalg.pinvh instead.
  warnings.warn(msg, category=DeprecationWarning)
/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function pinvh is deprecated; sklearn.utils.extmath.pinvh was deprecated in version 0.19 and will be removed in 0.21. Use scipy.linalg.pinvh instead.
  warnings.warn(msg, category=DeprecationWarning)
/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: Deprecat

66
0.0028395333133259437
380
0.0035309639150432623


/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function pinvh is deprecated; sklearn.utils.extmath.pinvh was deprecated in version 0.19 and will be removed in 0.21. Use scipy.linalg.pinvh instead.
  warnings.warn(msg, category=DeprecationWarning)
/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function pinvh is deprecated; sklearn.utils.extmath.pinvh was deprecated in version 0.19 and will be removed in 0.21. Use scipy.linalg.pinvh instead.
  warnings.warn(msg, category=DeprecationWarning)
/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function pinvh is deprecated; sklearn.utils.extmath.pinvh was deprecated in version 0.19 and will be removed in 0.21. Use scipy.linalg.pinvh instead.
  warnings.warn(msg, category=DeprecationWarning)
/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: Deprecat

67
0.0035309639150432623
67
0.0035309639150432623


/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function pinvh is deprecated; sklearn.utils.extmath.pinvh was deprecated in version 0.19 and will be removed in 0.21. Use scipy.linalg.pinvh instead.
  warnings.warn(msg, category=DeprecationWarning)
/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function pinvh is deprecated; sklearn.utils.extmath.pinvh was deprecated in version 0.19 and will be removed in 0.21. Use scipy.linalg.pinvh instead.
  warnings.warn(msg, category=DeprecationWarning)
/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function pinvh is deprecated; sklearn.utils.extmath.pinvh was deprecated in version 0.19 and will be removed in 0.21. Use scipy.linalg.pinvh instead.
  warnings.warn(msg, category=DeprecationWarning)
/scratch/xp_python3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: Deprecat

In [113]:
metrics.adjusted_mutual_info_score([1,1,1,0,0,0], [0,0,0,1,1,0])

0.3374678063574593